<a href="https://colab.research.google.com/github/Yutongzhang20080108/Using-a-Combination-of-DPO-and-RAG-to-Improve-the-Chinese-Poetry-Generation/blob/main/Test_between_Human_and_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_openai import ChatOpenAI

generate_model = ChatOpenAI(model="gpt-3.5-turbo", base_url="https://api.chatanywhere.tech/v1")
grade_model = ChatOpenAI(model="gpt-4", base_url="https://api.chatanywhere.tech/v1")

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
generate_system_template = "你是一个善于写中国古代诗歌的诗人，接下来我会给你提供一个题目，请你围绕这个题目写一首古诗。在生成的过程中请你注意中国古代诗歌的格式要求，并且请你表达尽可能多的情感与含义。在最后输出结果时只需要生成诗歌的主体部分"
generate_prompt_template = ChatPromptTemplate.from_messages(
    [("system", generate_system_template), ("user", "{text}")]
)

grade_system_template = "你是一个中国古代诗歌评论家，接下来我会给你两首诗歌，请你依据以下标准来评价这两首诗歌的得分。评分标准为：①格式。生成的唐诗是否遵循写作规则,尤其注意对仗、句数、字数、平仄（0至5分）；②创新性。生成的唐诗是否抄袭现成的诗句，是否具有创新性（0至5分）；③相关性。生成的唐诗与给定主题是否相关（0至5分）；④美学。生成的唐诗是否符合中国诗词的审美，包括寓情于物等（0至5分）；⑤整体。从整体情况评价生成的唐诗（0至10分）.请你在最后给出这几项的得分，并且你不需要给出评判的过程和任何文字只需要输出最后的5个数字。"
grade_prompt_template = ChatPromptTemplate.from_messages(
    [("system", grade_system_template), ("user", "{text}")]
)

In [5]:
generate_chain = generate_prompt_template | generate_model | parser
grade_chain = grade_prompt_template | grade_model | parser

In [7]:
import json
with open("/content/唐诗三百首.json", encoding="utf-8") as f:
  original_poetry = json.load(f)
#print(original_poetry[0])
original_text = []
for each in original_poetry:
  text = each["paragraphs"]
  text_all = ""
  for one in text:
    text_all = text_all + one
  original_text.append(text_all)

In [8]:
pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 7.9 MB/s eta 0:00:00


In [9]:
from opencc import OpenCC
cc = OpenCC('t2s')
def convert(text):
  converted = cc.convert(text)
  return convert

In [10]:
original_text_converted = []
for each in original_text:
  one = cc.convert(each)
  original_text_converted.append(one)

In [12]:
theme = []
for each in original_poetry:
  title = each["title"]
  theme.append(cc.convert(title))

In [68]:
generated_text = []
generated_score = []
for i in range(100):
   poem = generate_chain.invoke(f"请你围绕{theme[i]}这个题目写一首古诗")
   grade = grade_chain.invoke(f"{poem}")
   generated_text.append(poem)
   generated_score.append(grade)
print(generated_text)
print(generated_score)

['登高望远思难穹，\\\n岳咏蝉鸣动心中。\\\n夜深人静听声断，\\\n悠扬婉转入梦中。', '幽州台高远，登临望秋色。\n烟波浩渺间，思绪如飞絮。\n山川皆画境，云霞共舞曲。\n心随风飘荡，何处是归宿。', '昔日先贤孔子明，经邹鲁祭感怀情。\n百年风雨传佳话，千古遗风仰圣贤。\n凝思往事泪沾袖，铭记功德心永远。\n教化苍生传不朽，千古丰碑永流传。', '青袍红袖舞将进，醉卧花间思绪深。\n琴韵悠扬酒尚温，一曲高歌情不禁。\n金樽对酒心似火，豪情万丈意何极。\n风吹杨柳醉眼瞳，浮生若梦醉中寻。', '黄河远上白云间，横吹曲辞出塞关。\n万里戈声吹戍急，孤城孤驿望乡关。\n胡尘暮色笼平野，战马踏雪渡寒山。\n忆君心事殷勤问，愿将边疆报平安。', '黄河曲曲出塞地，胡马横行万里间。\n烽火台边烽烟起，将军独立望乡关。\n沙场战鼓齐鸣响，刀枪如雨血染山。\n忆往昔，边烽火；思归人，泪沾巾。\n今朝岁月今何在，烽火犹燃塞外天。\n愿得安邦归故里，不再横吹曲辞传。', '关山月，横吹曲辞，\n独倚栏杆思往事。\n寒风吹散忧愁意，\n皎皎明月照我心。\n\n山川万里情无限，\n江湖风波梦自深。\n且将心事随风去，\n任月光洒满山阴。', '婚礼喜临门，新嫁娘踏红被。\n花烛辉煌夜，笑语绕耳畔。\n亲朋皆祝福，良人情意切。\n共结良缘牢，白首不分离。', '故行宫，古木参天碧瓦红，\n往事如烟，落霞映照残阳。\n殿宇荒凉，风吹落叶满庭廊，\n寂寞凝望，燕归无人诉离伤。\n往昔繁华，今成空谷幽长，\n凝望远方，泪洒断桥残梦乡。', '锦瑟悠扬古调清，缠绵低吟入夜情。\n弦断音绝泪沾袖，翩跹舞动思绪生。\n长夜寂寞凝霜雪，孤灯照影泪漫流。\n梦中锦瑟如花落，凄凉离恨断肠愁。', '碧树枝头唤晓蝉，清风徐来叶欲翻。\n声声绕梁思无穷，千古传颂韵犹存。\n岁月流转似轮转，何惧芳华转眼繁。\n但愿人间能长在，与蝉共舞世间恩。', '青山翠竹绕乐游，溪水潺潺草萋萋。\n林间鸟语韵琴韵，花间风起舞幽幽。\n登高一望云蒸霞，心随飞鸟意无羁。\n且将闲逸留心间，与山水共乐游行。', '夜深人静雨潺潺，\n寄情北国思难言。\n孤灯一盏照窗外，\n泪滴无声滋心田。\n\n江水东流似故人，\n相逢何处共悲辛。\n愁云渡口难相见，\n独守寒窗待明晨。', '韩碑高耸入云天，\n千古传扬史载传。\n铭

In [14]:
human_score = []
for i in range(100):
  grade = grade_chain.invoke(f"{original_text_converted[i]}")
  human_score.append(grade)
print(human_score)

['4, 4, 4, 4, 8', '5 5 5 5 10', '4 4 4 4 8', '5 2 5 5 8', '5 5 5 5 10', '5 5 4 5 9', '5 5 5 5 9', '3.5, 3.5, 3, 4, 7.5', '5 5 5 5 10', '5 5 4 5 9', '4 4 3 4 8', '5 5 5 5 10', '5 5 5 5 10', '5 5 5 5 10', '4, 4, 5, 4, 8', '4 4 4 4 8', '5 5 5 5 10', '4 4 3 4 8', '5 5 4 5 9', '4.5, 5, 5, 4.5, 9', '4,5,4,5,8', '4 4 5 4 8', '5 5 5 5 10', '4 4 3 4 8', '5 4.5 4 4.5 8.5', '4, 4, 5, 4, 8', '5 4 5 5 9', '5, 5, 5, 5, 9', '4, 5, 3, 4, 7', '5 5 4 5 9', '4.5 4.0 4.0 4.5 8.5', '4 3 4 4 8', '4,4,4,4,8', '4, 4, 3, 4, 8', '4.5 4 4.5 4.5 9', '4 5 4 4 8', '4,4,4,4,8', '5 5 4 5 9', '4 4 4 4 8', '4,4,4,4,8', '4 4 5 4 8', '4, 4, 5, 4, 8', '4 4 3 3 7', '4 4 4 4 8', '5 4 4 4 8', '4, 3, 3, 4, 7', '5 4 4 5 8', '5 5 5 5 9', '4 3 4 4 8', '5 5 5 5 10', '5 5 5 5 10', '5 4 4 5 9', '4 4 4 4 8', '5 5 5 5 10', '5 4 4 4 8', '5 4 5 5 9', '5,4,5,4,8', '5 4 5 4 8', '5 5 5 5 9', '4 4 3 4 8', '4 4 4 4 8', '5 4 4 5 8', '5 5 4 5 9', '4 4 3 4 7', '4 4 3 4 7', '5 5 5 4 9', '4, 5, 4, 4, 7', '4 4 4 4 8', '4.5 4.5 4 4.5 8.5', '4,4,3,